In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Installing libraries

In [2]:
!pip install textattack
!pip install -U gensim==4.0.0
!pip install python-Levenshtein

     |████████████████████████████████| 358kB 10.6MB/s 
     |████████████████████████████████| 778kB 20.1MB/s 
     |████████████████████████████████| 102kB 11.3MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 2.1MB 34.3MB/s 
     |████████████████████████████████| 337kB 36.5MB/s 
     |████████████████████████████████| 20.1MB 5.4MB/s 
     |████████████████████████████████| 225kB 42.6MB/s 
     |████████████████████████████████| 3.3MB 36.3MB/s 
     |████████████████████████████████| 901kB 32.7MB/s 
     |████████████████████████████████| 983kB 33.7MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 19.7MB 1.4MB/s 
     |████████████████████████████████| 798kB 33.6MB/s 
     |████████████████████████████████| 1.2MB 33.0MB/s 
     |████████████████████████████████| 245kB 45.5MB/s 
     |████████████████████████████████| 112kB 38.8MB/s 
  Created wheel for terminaltables: filename=termina

     |████████████████████████████████| 23.9MB 131kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 51kB 4.6MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149800 sha256=6c31e73150df6dec1a9454b820ef740e5111da332c2a7c39a392e54ff4a6ab58
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [3]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
from gensim.models import Phrases
from gensim.test.utils import get_tmpfile
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.transformations import WordSwap
from textattack.search_methods import GreedySearch, GreedyWordSwapWIR
from textattack.constraints.overlap import LevenshteinEditDistance
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:21<00:00, 22.6MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpxqzt6w34.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [ ]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.fasttext import FastText
# model1 = FastText(corpus, sample = 0, sg = 1)

In [ ]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.word2vec import Word2Vec
# model2 = Word2Vec(corpus, sample = 0, sg = 1)

# Loading FastText, Word2Vec and Glove

In [4]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
#fname = get_tmpfile("/models/fasttext.model") 
model1 = FastText.load("/content/drive/MyDrive/models/fasttext.model") # Change this location

In [5]:
from gensim.test.utils import get_tmpfile
from gensim.models.word2vec import Word2Vec
#fname1 = get_tmpfile("/models/word2vec.model") 
model2 = Word2Vec.load("/content/drive/MyDrive/models/word2vec.model") # Change this location

In [6]:
from gensim.models import KeyedVectors
model3=KeyedVectors.load('/content/drive/MyDrive/models/gloveKeyVectors.kv')

Example demonstration

In [7]:
print(model1.wv.most_similar('cat'))
print(model2.wv.most_similar('cat'))
print(model3.most_similar('cat'))

[('bobcat', 0.759185791015625), ('dog', 0.7208123803138733), ('foxglove', 0.7110776901245117), ('pig', 0.7088143229484558), ('goose', 0.7069101333618164), ('cats', 0.702387273311615), ('beak', 0.7008379697799683), ('catfish', 0.6987316608428955), ('foxhound', 0.696578860282898), ('catnip', 0.6960986852645874)]
[('dog', 0.7193913459777832), ('squirrel', 0.696162760257721), ('leopardus', 0.6945451498031616), ('prionailurus', 0.688106894493103), ('albino', 0.6880608201026917), ('bird', 0.6836060285568237), ('eared', 0.6803243160247803), ('rhinoceros', 0.6769237518310547), ('hyena', 0.6756084561347961), ('badger', 0.6752869486808777)]
[('cats', 0.7006820440292358), ('dog', 0.698471188545227), ('dogs', 0.6442825198173523), ('tervueren', 0.619421660900116), ('laekenois', 0.5972026586532593), ('lica', 0.5956287980079651), ('feral', 0.5899650454521179), ('goat', 0.5861616134643555), ('fish', 0.5746133327484131), ('malinois', 0.5737614631652832)]


In [8]:
if 'cat' in model3.key_to_index:
  print('present')

present


# Distilbert Base Cased-QQP

In [9]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/distilbert-base-cased-SST-2")
tokenizer = AutoTokenizer("textattack/distilbert-base-cased-SST-2")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Framing fasttext attack

In [10]:
from textattack.transformations import WordSwap

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

In [11]:
def attack_stats(attack, dataset, num_of_examples=25):
  print(attack)
  num_examples = num_of_examples
  num_remaining_attacks = num_examples
  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

  worklist = deque(range(0, num_examples))
  worklist_tail = worklist[-1]

  attack_log_manager = textattack.loggers.AttackLogManager()

  load_time = time.time()
        
  num_results = 0
  num_failures = 0
  num_successes = 0
  for result in attack.attack_dataset(dataset, indices=worklist):
      result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
      display(HTML(result_html_str))
      attack_log_manager.log_result(result)
            
      if not isinstance(result, textattack.attack_results.SkippedAttackResult):
          pbar.update(1)
      else:
          worklist_tail += 1
          pbar.update(1)
          worklist.append(worklist_tail)

      num_results += 1

      if (type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult):
          num_successes += 1
                
      if type(result) == textattack.attack_results.FailedAttackResult:
          num_failures += 1
      pbar.set_description(
          "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
      )

  pbar.close()

  attack_log_manager.enable_stdout()
  attack_log_manager.log_summary()
  attack_log_manager.flush()
        
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

### FastText, GreedySearch, RepeatModification, StopwordModification

In [12]:

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("glue", "sst2", "validation") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


textattack: Loading datasets dataset glue, subset sst2, split validation.


Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [13]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:15,  1.56it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:09,  2.40it/s]

[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:01<00:14,  1.51it/s]

...

[Succeeded / Failed / Total] 7 / 16 / 25: 100%|██████████| 25/25 [00:25<00:00,  1.01s/it]

[Succeeded / Failed / Total] 7 / 17 / 26: : 26it [00:29,  1.13s/it]                      

[Succeeded / Failed / Total] 7 / 18 / 27: : 27it [00:32,  1.19s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 18     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 66.67% |
| Attack success rate:          | 28.0%  |
| Average perturbed word %:     | 17.98% |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 49.16  |
+-------------------------------+--------+


textattack: Attack time: 32.19425129890442s


# Framing word2vec attack

In [14]:
class Swapper_(WordSwap): # For word2vec
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model2.wv.key_to_index:
            for i in range(10):
                if word.lower() in model2.wv.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model2.wv.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model2.wv.most_similar(word)[i][0].capitalize()]
                else:
                    return [model2.wv.most_similar(word)[i][0]]
        else:
            return [word]

### Word2vec, GreedySearch, RepeatModification, StopwordModification

In [15]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [16]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [17]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:04,  4.93it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:03,  6.39it/s]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:02<00:15,  1.41it/s]

...

[Succeeded / Failed / Total] 10 / 13 / 25: 100%|██████████| 25/25 [00:15<00:00,  1.65it/s]

[Succeeded / Failed / Total] 11 / 13 / 26: : 26it [00:17,  1.49it/s]                      

[Succeeded / Failed / Total] 11 / 14 / 27: : 27it [00:19,  1.37it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 11     |
| Number of failed attacks:     | 14     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 51.85% |
| Attack success rate:          | 44.0%  |
| Average perturbed word %:     | 14.23% |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 33.56  |
+-------------------------------+--------+


textattack: Attack time: 19.693322896957397s


### Framing Glove attack

In [18]:
from textattack.transformations import WordSwap

class Swapper__(WordSwap): # For glove
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model3.key_to_index:
            for i in range(10):
                if word.lower() in model3.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model3.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model3.most_similar(word)[i][0].capitalize()]
                else:
                    return [model3.most_similar(word)[i][0]]
        else:
            return [word]

### Glove, GreedySearch, RepeatModification, StopwordModification

In [19]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [20]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [21]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:08,  2.82it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:07,  2.93it/s]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:04<00:30,  1.39s/it]

...

[Succeeded / Failed / Total] 15 / 8 / 25: 100%|██████████| 25/25 [00:24<00:00,  1.01it/s]

[Succeeded / Failed / Total] 16 / 8 / 26: : 26it [00:26,  1.03s/it]                      

[Succeeded / Failed / Total] 16 / 9 / 27: : 27it [00:31,  1.17s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 16     |
| Number of failed attacks:     | 9      |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 33.33% |
| Attack success rate:          | 64.0%  |
| Average perturbed word %:     | 13.6%  |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 27.2   |
+-------------------------------+--------+


textattack: Attack time: 31.485305309295654s


### FastText, GreedyWordSwapWIR, RepeatModification, StopwordModification

In [22]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [23]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [24]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:06,  3.57it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:00<00:05,  4.31it/s]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:01<00:08,  2.47it/s]

...

[Succeeded / Failed / Total] 5 / 18 / 25: 100%|██████████| 25/25 [00:12<00:00,  1.94it/s]

[Succeeded / Failed / Total] 5 / 19 / 26: : 26it [00:13,  1.86it/s]                      

[Succeeded / Failed / Total] 5 / 20 / 27: : 27it [00:14,  1.81it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 20     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 74.07% |
| Attack success rate:          | 20.0%  |
| Average perturbed word %:     | 15.66% |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 25.96  |
+-------------------------------+--------+


textattack: Attack time: 14.958003997802734s


### Word2Vec, GreedyWordSwapWIR, RepeatModification, StopwordModification

In [25]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [26]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [27]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:05,  4.46it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:04,  5.11it/s]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:01<00:08,  2.66it/s]

...

[Succeeded / Failed / Total] 10 / 13 / 25: 100%|██████████| 25/25 [00:11<00:00,  2.20it/s]

[Succeeded / Failed / Total] 11 / 13 / 26: : 26it [00:12,  2.11it/s]                      

[Succeeded / Failed / Total] 11 / 14 / 27: : 27it [00:13,  2.03it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 11     |
| Number of failed attacks:     | 14     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 51.85% |
| Attack success rate:          | 44.0%  |
| Average perturbed word %:     | 17.8%  |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 24.36  |
+-------------------------------+--------+


textattack: Attack time: 13.287036657333374s


### Glove, GreedyWordSwapWIR RepeatModification, StopwordModification

In [28]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [29]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [30]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:06,  3.73it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:06,  3.83it/s]

[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:01<00:10,  2.03it/s]

...

[Succeeded / Failed / Total] 16 / 7 / 25: 100%|██████████| 25/25 [00:13<00:00,  1.92it/s]

[Succeeded / Failed / Total] 17 / 7 / 26: : 26it [00:14,  1.84it/s]                      

[Succeeded / Failed / Total] 17 / 8 / 27: : 27it [00:15,  1.75it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 17     |
| Number of failed attacks:     | 8      |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 29.63% |
| Attack success rate:          | 68.0%  |
| Average perturbed word %:     | 18.48% |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 22.96  |
+-------------------------------+--------+


textattack: Attack time: 15.398909091949463s


### FastText, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [31]:
from textattack.search_methods import ParticleSwarmOptimization


# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [32]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [33]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:13,  1.78it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:08,  2.62it/s]

[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:06<00:46,  2.10s/it]

...

[Succeeded / Failed / Total] 7 / 16 / 25: 100%|██████████| 25/25 [01:31<00:00,  3.65s/it]

[Succeeded / Failed / Total] 7 / 17 / 26: : 26it [01:50,  4.23s/it]

[Succeeded / Failed / Total] 7 / 18 / 27: : 27it [01:58,  4.39s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 18     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 66.67% |
| Attack success rate:          | 28.0%  |
| Average perturbed word %:     | 22.34% |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 257.08 |
+-------------------------------+--------+


textattack: Attack time: 118.46835923194885s


### Word2Vec, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [34]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [35]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [36]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:08,  2.84it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:07,  2.92it/s]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:11<01:21,  3.70s/it]

...

[Succeeded / Failed / Total] 10 / 13 / 25: 100%|██████████| 25/25 [00:53<00:00,  2.13s/it]

[Succeeded / Failed / Total] 11 / 13 / 26: : 26it [01:10,  2.69s/it]

[Succeeded / Failed / Total] 11 / 14 / 27: : 27it [01:11,  2.63s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 11     |
| Number of failed attacks:     | 14     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 51.85% |
| Attack success rate:          | 44.0%  |
| Average perturbed word %:     | 16.41% |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 182.84 |
+-------------------------------+--------+


textattack: Attack time: 71.10034513473511s


### Glove, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [37]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [38]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [39]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:13,  1.73it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:01<00:12,  1.88it/s]

[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:16<01:59,  5.45s/it]

...

[Succeeded / Failed / Total] 16 / 7 / 25: 100%|██████████| 25/25 [01:29<00:00,  3.60s/it]

[Succeeded / Failed / Total] 17 / 7 / 26: : 26it [01:30,  3.47s/it]

[Succeeded / Failed / Total] 17 / 8 / 27: : 27it [01:43,  3.82s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 17     |
| Number of failed attacks:     | 8      |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 29.63% |
| Attack success rate:          | 68.0%  |
| Average perturbed word %:     | 18.53% |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 143.76 |
+-------------------------------+--------+


textattack: Attack time: 103.02463483810425s


### FastText, GreedyWordSwapWIR, PartsofSpeech

In [43]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [44]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [45]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:09,  2.42it/s]


[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:00<00:07,  3.24it/s]


[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:01<00:10,  2.00it/s]

...


[Succeeded / Failed / Total] 7 / 16 / 25: 100%|██████████| 25/25 [00:16<00:00,  1.56it/s]


[Succeeded / Failed / Total] 7 / 17 / 26: : 26it [00:17,  1.51it/s]                      


[Succeeded / Failed / Total] 7 / 17 / 26: : 27it [00:17,  1.53it/s]
[Succeeded / Failed / Total] 8 / 17 / 27: : 27it [00:17,  1.52it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 62.96% |
| Attack success rate:          | 32.0%  |
| Average perturbed word %:     | 26.96% |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 27.04  |
+-------------------------------+--------+


textattack: Attack time: 17.74280333518982s


### Word2Vec, GreedyWordSwapWIR, PartsofSpeech

In [46]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [47]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [48]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:09,  2.60it/s]


[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:00<00:06,  3.65it/s]


[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:01<00:10,  2.13it/s]

...


[Succeeded / Failed / Total] 7 / 16 / 25: 100%|██████████| 25/25 [00:14<00:00,  1.77it/s]


[Succeeded / Failed / Total] 7 / 17 / 26: : 26it [00:15,  1.71it/s]                      


[Succeeded / Failed / Total] 7 / 17 / 26: : 27it [00:15,  1.72it/s]
[Succeeded / Failed / Total] 8 / 17 / 27: : 27it [00:15,  1.72it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 62.96% |
| Attack success rate:          | 32.0%  |
| Average perturbed word %:     | 13.88% |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 25.2   |
+-------------------------------+--------+


textattack: Attack time: 15.725313425064087s


### Glove, GreedyWordSwapWIR, PartsofSpeech

In [49]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [50]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [51]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:14,  1.62it/s]


[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:00<00:10,  2.24it/s]


[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:02<00:16,  1.33it/s]

...


[Succeeded / Failed / Total] 9 / 14 / 25: 100%|██████████| 25/25 [00:19<00:00,  1.26it/s]


[Succeeded / Failed / Total] 9 / 15 / 26: : 26it [00:21,  1.22it/s]                      


[Succeeded / Failed / Total] 9 / 15 / 26: : 27it [00:21,  1.23it/s]
[Succeeded / Failed / Total] 10 / 15 / 27: : 27it [00:21,  1.23it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 15     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 55.56% |
| Attack success rate:          | 40.0%  |
| Average perturbed word %:     | 13.38% |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 23.6   |
+-------------------------------+--------+


textattack: Attack time: 21.926517963409424s


### FastText, ParticleSwarmOptimization, PartsofSpeech

In [52]:
from textattack.constraints.grammaticality import PartOfSpeech
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [53]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [54]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:01<00:32,  1.35s/it]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:01<00:20,  1.14it/s]


[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:11<01:22,  3.73s/it]

...


[Succeeded / Failed / Total] 14 / 9 / 25: 100%|██████████| 25/25 [02:48<00:00,  6.72s/it]


[Succeeded / Failed / Total] 14 / 9 / 25: : 26it [03:09,  7.30s/it]                      
[Succeeded / Failed / Total] 14 / 10 / 26: : 26it [03:09,  7.30s/it]


[Succeeded / Failed / Total] 14 / 10 / 26: : 27it [03:10,  7.05s/it]
[Succeeded / Failed / Total] 15 / 10 / 27: : 27it [03:10,  7.05s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 15     |
| Number of failed attacks:     | 10     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 37.04% |
| Attack success rate:          | 60.0%  |
| Average perturbed word %:     | 24.44% |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 318.6  |
+-------------------------------+--------+


textattack: Attack time: 190.43144178390503s


### Word2Vec, ParticleSwarmOptimization, PartsofSpeech

In [55]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [56]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [57]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:01<00:43,  1.81s/it]


[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:02<00:23,  1.02s/it]


[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:13<01:36,  4.39s/it]

...


[Succeeded / Failed / Total] 13 / 10 / 25: 100%|██████████| 25/25 [02:33<00:00,  6.14s/it]


[Succeeded / Failed / Total] 13 / 10 / 25: : 26it [02:45,  6.35s/it]                      
[Succeeded / Failed / Total] 13 / 11 / 26: : 26it [02:45,  6.35s/it]


[Succeeded / Failed / Total] 13 / 11 / 26: : 27it [02:45,  6.14s/it]
[Succeeded / Failed / Total] 14 / 11 / 27: : 27it [02:45,  6.14s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 14     |
| Number of failed attacks:     | 11     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 40.74% |
| Attack success rate:          | 56.0%  |
| Average perturbed word %:     | 19.45% |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 290.6  |
+-------------------------------+--------+


textattack: Attack time: 165.748841047287s


### Glove, ParticleSwarmOptimization, PartsofSpeech

In [58]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [59]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [60]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:05<02:14,  5.60s/it]


[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:06<01:14,  3.26s/it]


[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:33<04:02, 11.02s/it]

...


[Succeeded / Failed / Total] 11 / 12 / 25: 100%|██████████| 25/25 [04:18<00:00, 10.34s/it]


[Succeeded / Failed / Total] 11 / 12 / 25: : 26it [04:53, 11.29s/it]                      
[Succeeded / Failed / Total] 11 / 13 / 26: : 26it [04:53, 11.29s/it]


[Succeeded / Failed / Total] 11 / 13 / 26: : 27it [04:54, 10.92s/it]
[Succeeded / Failed / Total] 12 / 13 / 27: : 27it [04:54, 10.92s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 12     |
| Number of failed attacks:     | 13     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 48.15% |
| Attack success rate:          | 48.0%  |
| Average perturbed word %:     | 15.25% |
| Average num. words per input: | 16.33  |
| Avg num queries:              | 229.36 |
+-------------------------------+--------+


textattack: Attack time: 294.806645154953s
